In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate

In [ ]:
bs = 16
path = Path('../data/state-farm-distracted-driver-detection/')

In [ ]:
data = (
    ImageDataBunch.from_folder(path, train='train', test='test', valid_pct=0.2,
                               classes=['c0', 'c1', 'c2', 'c3', 'c4', 
                                        'c5', 'c6', 'c7', 'c8', 'c9'], 
                               ds_tfms=get_transforms(), size=224, bs=bs).
    normalize(imagenet_stats)
)

In [ ]:
data.show_batch(rows=3, figsize=(15, 15))

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.test_ds)

In [ ]:
data.test_ds

In [ ]:
data.train_ds

In [ ]:
learner = cnn_learner(data, models.resnet50, metrics=[error_rate, accuracy])

In [ ]:
learner.fit_one_cycle(4)

In [ ]:
learner.save('sf-distdriver-stage-1')

In [ ]:
learner.load('sf-distdriver-stage-1')

In [ ]:
learner.unfreeze()

In [ ]:
learner.purge()

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)

In [ ]:
interp.plot_top_losses(10)

In [ ]:
learner.lr_find()

In [ ]:
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(2, max_lr=slice(1e-06, 1e-05))

In [ ]:
learner.save('sf-distdriver-stage-2')

In [ ]:
preds, y = learner.get_preds(DatasetType.Test)

In [ ]:
preds

In [ ]:
def gen_sub(preds, data, filepath='../submission-statefarm-distdriver.csv',
            low=.02, high=.98):
    proba = [p for p in (np.clip(preds, low, high))]
    fname = [str(i).split('/')[-1] for i in data.test_dl.dataset.items]
    
    with open(filepath, 'w') as f:
        f.write('img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9\n')
        for i, file in enumerate(fname):
            f.write(file + ',')
            for j, val in enumerate(proba[i]):
                if j == len(proba[i]) - 1:
                    f.write(str(np.round(np.float(val), 3)) + '\n')
                else:
                    f.write(str(np.round(np.float(val), 3)) + ',')

In [ ]:
gen_sub(preds, data, low=.05, high=.95)

In [ ]:
[str(i).split('/')[-1] for i in data.test_dl.dataset.items]

In [ ]:
preds

In [ ]:
preds[0]

In [ ]:
foo = pd.read_csv('../submission-statefarm-distdriver.csv')

In [ ]:
len(foo.columns)

In [ ]:
[str(i).split('/')[-1] for i in data.test_dl.dataset.items]